In [1]:
# Importint Relevant Libraries
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

There are a varied number of options that you can configure when setting up a SparkSession. Let's go over a few of the more common ones:
* **master**: The URL for the cluster SparkContext to connect to master: The URL for the cluster SparkContext to connect to
* **appName**: The name that will be displayed in the Spark cluster UI
* **config**: Configuration for SparkSession. Any key-value pairs in the config will be applied to the session's SparkConf. For example, you can set the spark.sql.shuffle.partitions configuration property to change the number of partitions in joins and aggregations. Or you can set spark.executor.memory to change the amount of memory used per executor process.

In [5]:
# Creating a Spark Session
# Wrapping the session in brackets allows us to chain commands without using a "\" in Python
spark = (SparkSession.builder
        .master("local[*]") # Using all cores
        .appName("Catch-Up Session")
        .getOrCreate())

In [6]:
# Viewing the Spark Session
spark

The SparkSession is the entry point to Spark SQL. It manages the SparkContext that was used to create it, and provides a way to create DataFrames and DataSets. Spark SQL is the Spark module for working with structured data. It allows you to use SQL or the DataFrame/Dataset API to express Spark operations on structured data.

For this exercise, we will be going over how we can read, access and manipulate structured data in Spark using Spark SQL. The data we will be using will be stored in the data directory contained in this repository